## Demonstrate the use of the make_angles_image

You will need to install [geemap](https://github.com/giswqs/geemap) to visualize the results of this example. 

In [1]:
import ee 
ee.Initialize()

In [2]:
# Select a cloudy Sentinel2 image
date_str='2017-04-15' 
lonlat=[38.530867, 30.236383] 
date = ee.Date(date_str)
geom = ee.Geometry.Point(lonlat[0], lonlat[1]) # lon/lat

S2_img = ee.Image(
  ee.ImageCollection('COPERNICUS/S2')
    .filterBounds(geom)
    .filterDate(date,date.advance(3,'month'))
    .sort('system:time_start')
    .first()
  )

In [3]:
# The metadata file's url of the image hosted on google cloud storage can be obtained as follows:
meta_info = S2_img.getInfo()
meta_bands = meta_info['bands']
meta_prop = meta_info['properties']
info_procLevel, info_tileId = tuple(meta_prop['GRANULE_ID'].split('_')[0:2])

gs_prefix = 'gs://gcp-public-data-sentinel-2/tiles'
gs_prefix= 'https://storage.googleapis.com/gcp-public-data-sentinel-2/tiles'
BASE_URL = '{}/{}/{}/{}/{}'.format(gs_prefix, info_tileId[1:3], info_tileId[3:4], info_tileId[4:6], meta_prop['PRODUCT_URI'])
PATH_XML = '{}/GRANULE/{}/MTD_TL.xml'.format(BASE_URL, meta_prop['GRANULE_ID'])
PATH_XML2 ='{}/MTD_MSIL1C.xml'.format(BASE_URL)
print(PATH_XML)

https://storage.googleapis.com/gcp-public-data-sentinel-2/tiles/37/R/DP/S2A_MSIL1C_20170415T080001_N0204_R035_T37RDP_20170415T080445.SAFE/GRANULE/L1C_T37RDP_A009470_20170415T080445/MTD_TL.xml


In [4]:
# Call sentinel2makeAnglesImage to get the angles image
from fmask.cmdline import sentinel2makeAnglesImage
angles_img = sentinel2makeAnglesImage.makeAngles_ee(PATH_XML)

# Visualization of a band in the angles image in geemap:
# available bands are: SatelliteAzimuth, SatelliteZenith, SunAzimuth, SunZenith
import geemap
Map = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=10)
Map.addLayer(angles_img, {'bands':'SatelliteAzimuth','min':150, 'max':223, 'palette':["#440154","#3a528b","#20908d","#5dc962","#fde725"], 'opacity': 0.8})
Map

Map(center=[30.236383, 38.530867], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…